# Subset Selection
* approach involves identifying a subset of p features (predictors) out of k that we think are related to the response

* then fit a model using OLS on the reduced set of variables

In [5]:
import pandas as pd
import numpy as np
import patsy
import itertools
import time
import statsmodels.api as sm

# import dataset 
hprice2 = pd.read_stata('http://fmwww.bc.edu/ec-p/data/wooldridge/hprice2.dta')

# write specification
f = 'lprice ~ lnox + lproptax + crime + rooms + dist + radial + stratio + lowstat'

# create design matrices
y1, X1 = patsy.dmatrices(f, data=hprice2, return_type='dataframe')

In [6]:
# pre-processing: demean outcome and features so all models can be fitted without an intercept
y = y1.sub(y1.mean())
X = X1.sub(X1.mean()).drop('Intercept',axis=1)

## Best Subset Selection
* fit separate OLS regression best subset for each possible combination of k predictors

* fit all k models that contain exactly one predictor, then all ${k \choose 2} = \frac{k(k-1)}{2}$ models that contain exactly two predictors, etc.

* look at resulting models to identify the best one

* number of models to consider grows rapidly as k increases!

### Algorithm: Best Subset Selection

1. Let $\mathcal{M_0}$ denote null model (i.e no predictors) - predicts sample mean for each observation

2. For p = 1,2,..., k:

a) Fit all ${k \choose p}$ models that contain exactly p predictors

b) Pick best among these ${k \choose p}$ models and call it $\mathcal{M_p}$; in this case, best is defined as having the smallest RSS or, equivalently, the largest $R^2$

In [7]:
# define a function that takes the predictors selected and subsets the X design matrix
def processSubset(feature_set):
    # Fit model on feature_set
    model = sm.OLS(y,X[list(feature_set)])
    regr = model.fit()

    # calculate RSS
    RSS = regr.ssr
    return {"model":regr, "RSS":RSS}

# define a function that selects the best model with p number of predictors
def getBest(p):
    
    # empty list to collect model and RSS 
    results = []
    
    # iterate through the different predictor combinations subject to a limit of p predictors
    for combo in itertools.combinations(X.columns, p):
        results.append(processSubset(combo))
    
    # Create a dataframe of the results
    models = pd.DataFrame(results)
    
    # Choose the model with the lowest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    # Return the best model, along with some other useful information about the model
    return best_model

# dataframe where best models will be collected
models_best = pd.DataFrame(columns=["RSS", "model"])

# for loop that collects the best model for each number of predictors
for i in range(1,9):
    models_best.loc[i] = getBest(i)

In [14]:
# make graphs of results - will do later

## Stepwise Selection: Forward Stepwise Selection
* computational reasoning, previous 'best subset selection' can't be applied with very large k's

* this algorithm begins with a model containing no predictors, then adds predictors to the model one-at-a-time until all predictors are in the model

### Algorithm: Forward Stepwise Selection

1. Let $\mathcal{M_0}$ denote the null model which contains no predictors

2. For p = 0,1,2,..., k-1

A. Consider all k-p models that augment the predictors in $\mathcal{M_p}$ with one additional predictor

B. Choose the best among these k-p models; call it $\mathcal{M_{p+1}}$; best is defined as having smallest RSS or highest $R^2$

3. Then select a single best model form

In [ ]:
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    
    # create an empty list
    results = []
    
    # take each p in the remaining predictors and create a model & take RSS
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
    
    # collect models in a data frame
    models = pd.DataFrame(results)
    
    # Choose the model with the lowest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    # Return the best model, along with some other useful information about the model
    return best_model

models_fwd = pd.DataFrame(columns=["RSS", "model"])

predictors = []

for i in range(1,len(X.columns)+1):    
    models_fwd.loc[i] = forward(predictors)
    predictors = models_fwd.loc[i]["model"].model.exog_names

## Stepwise Selection: Backward Stepwise Selection
* begins with full least squares model containing all k regressors

* then iteractively remove the least useful regressors one-at-a-time until there are no regressors in the model

### Algorithm: Backward Stepwise Selection

1. Let $\mathcal{M_k}$ deonte the full model that contains all k predictors

2. For p = k, k-1, ..., 1

A. Consider all p models that contain all but one of the predictors in $\mathcal{p}$ for a total of p-1 predictors

B. Choose the best model among these p models; call it $\mathcal{M_{p-1}}$; best defined as having smallest RSS or highest $R^2$

3. Select single best model from among $\mathcal{M_0},..., \mathcal{M_k}$ using a model selection criteria previously discussed

In [ ]:
# copied - need to look over code

def backward(predictors):

    results = []
    
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the lowest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    # Return the best model, along with some other useful information about the model
    return best_model

models_bwd = pd.DataFrame(columns=["RSS", "model"], index = range(1,len(X.columns)))

predictors = X.columns

while(len(predictors) > 1):  
    models_bwd.loc[len(predictors)-1] = backward(predictors)
    predictors = models_bwd.loc[len(predictors)-1]["model"].model.exog_names

## Choosing the Optimal Model
* previous algorithms: good at selecting a model that best fit the training data set (since they minimize the RSS which is equivalent to maximizing R^2 for these observations)

* does not necessarily mean they minimize the RSS in any validation set

* therefore, the training set RSS and the training set R2 cannot be used to select from among a set of models with different numbers of variables if we're using this model for prediction

* can replace step 3 in these algorithms with a step where the cross-validated prediction error is calculated using observations in the validation set

* can use other model selection criteria ($\bar{R}^2$, Cp, AIC, BIC, etc) but if you want to know about the prediction power of model, cross validation prediction error is more natural choice

In [ ]:
# copied - look over later

def processSubset(feature_set, X_train, y_train, X_test, y_test):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(y_train,X_train[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(X_test[list(feature_set)]) - y_test.iloc[:,0])**2).sum()
    return {"model":regr, "RSS":RSS}

def forward(predictors, X_train, y_train, X_test, y_test):
    results = []

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X_train.columns if p not in predictors]
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p], X_train, y_train, X_test, y_test))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the lowest RSS
    best_model = models.loc[models['RSS'].argmin()]
        
    # Return the best model, along with some other useful information about the model
    return best_model

# number of folds
k = 10
np.random.seed(seed=1)
folds = np.random.choice(k, size = len(y), replace = True)

# Create a DataFrame to store the results of our upcoming calculations
cv_errors = pd.DataFrame(columns=range(1,k+1), index=range(1,9))

models_cv = pd.DataFrame(columns=["RSS", "model"])

# Outer loop iterates over all folds
for j in range(1,k+1):

    # Reset predictors
    predictors = []
    
    # Inner loop iterates over each size i
    for i in range(1,len(X.columns)+1):    
    
        # The perform forward selection on the full dataset minus the jth fold, test on jth fold
        models_cv.loc[i] = forward(predictors,  X[folds != (j-1)], y[folds != (j-1)], X[folds == (j-1)], y[folds == (j-1)
        
        # Save the cross-validated error for this fold
        cv_errors[j][i] = models_cv.loc[i]["RSS"]

        # Extract the predictors
        predictors = models_cv.loc[i]["model"].model.exog_names

cv_errors

cv_mean = cv_errors.apply(np.mean, axis=1)

#extract the best model and estimate its parameters using the entire data set

print(sm.OLS(y1,sm.add_constant(X1[models_cv.loc[7, "model"].model.exog_names])).fit().summary())
print(sm.OLS(y,X[models_cv.loc[7, "model"].model.exog_names]).fit().summary())
